<a href="https://colab.research.google.com/github/krvzkk/NeuralNetworkk/blob/main/Proba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ai-forever/diffusers
!pip install /content/diffusers
!pip install transformers
!pip install accelerate
!pip install fastparquet

Cloning into 'diffusers'...
remote: Enumerating objects: 34834, done.
remote: Total 34834 (delta 0), reused 0 (delta 0), pack-reused 34834
Receiving objects: 100% (34834/34834), 28.03 MiB | 13.88 MiB/s, done.
Resolving deltas: 100% (24823/24823), done.
Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.24.0.dev0-py3-none-any.whl size=1688546 sha256=9a93b0c864c321c412b8d51a3523b45acf99c0b08125e5f7f7c927c3d5e0c632
  Stored in directory: /tmp/pip-ephem-wheel-cache-qybub5_i/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cac

In [9]:
%%capture
!wget https://huggingface.co/datasets/lambdalabs/pokemon-blip-captions/resolve/main/data/train-00000-of-00001-566cc9b19d7203f8.parquet


In [30]:
%%capture
!https://huggingface.co/datasets/MohamedRashad/midjourney-detailed-prompts/resolve/main/data/train-00000-of-00002.parquet?download=true

In [31]:
import pandas as pd
from PIL import Image
import io
import os
df_dataset = pd.read_parquet('https://huggingface.co/datasets/MohamedRashad/midjourney-detailed-prompts/resolve/main/data/train-00000-of-00002.parquet?download=true', engine='fastparquet')

In [32]:
df_dataset

,image_description,long_prompt,short_prompt,image.bytes,image.path
0,The image presents a captivating digital art p...,"An ethereal, detailed digital illustration of ...","A serene white wolf, its fur a rainbow kaleido...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,0000.jpg
1,"In the image, a small robot is standing in the...","A charming nighttime scene, a small robot with...","A colorful robot braves the storm, sheltering ...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,0001.jpg
2,The image portrays a young woman with vibrant ...,A detailed digital illustration of a young wom...,A portrait: Serene young woman with vibrant pi...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,0002.jpg
3,"The image portrays a majestic figure, clad in ...",A detailed oil painting: A robed figure stands...,"An ethereal, masked figure stands tall, staff ...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,0003.jpg
4,The image presents a striking scene of a lone ...,A detailed digital illustration of a resolute ...,"An epic, atmospheric landscape featuring a sol...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,0004.jpg
...,...,...,...,...,...
1522,The image presents a striking portrait of a wo...,A detailed digital illustration of a woman's p...,A close-up oil painting of a woman's face with...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,1534.jpg
1523,The image presents a captivating digital illus...,"An ethereal digital portrait, featuring a woma...",A digital portrait of a woman in blue and gold...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,1535.jpg
1524,The image presents a striking digital illustra...,"An ethereal, detailed digital illustration of ...","An ethereal, fiery tiger roars in the forefron...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,1536.jpg
1525,The image portrays a dramatic scene of a city ...,"A dramatic, high-resolution 3D render: an erup...",An ominous orange skyline frames a witness to ...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,1537.jpg


In [34]:
dir_name = '/content/data'
os.mkdir(dir_name)

In [37]:
df = []
for i in range(len(df_dataset)):
    image = Image.open(io.BytesIO(df_dataset['image.bytes'].iloc[i]))
    img_path = os.path.join(dir_name, f'{i}.jpg')
    image.save(img_path)
    df.append([img_path, df_dataset['long_prompt'].iloc[i]])
df = pd.DataFrame(df)
df.columns = ['paths', 'caption']
df.to_csv('captions.csv', index=False)

In [38]:
command = (f'python3 /content/diffusers/examples/kandinsky2_2_train/tune_decoder_lora.py '
           f'--train_images_paths_csv=/content/captions.csv '
           f'--image_resolution=768 '
           f'--train_batch_size=1 '
           f'--gradient_accumulation_steps=1 '
           f'--gradient_checkpointing '
           f'--mixed_precision="fp16"  '
           f'--max_train_steps=500 '
           f'--lr=1e-05 '
           f'--max_grad_norm=1 '
           f'--lr_scheduler="constant" '
           f'--lr_warmup_steps=0 '
           f'--output_dir=/content/decoder_lora_saves '
           f'--rank=4 '
           f'--checkpointing_steps=500')

In [39]:
!{command}

2024-05-08 14:58:35.880255: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 14:58:35.880312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 14:58:35.994622: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 14:58:37.061452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_

In [40]:
command = (f'python3 /content/diffusers/examples/kandinsky2_2_train/tune_prior_lora.py '
           f'--train_images_paths_csv=/content/captions.csv '
           f'--train_batch_size=1 '
           f'--gradient_accumulation_steps=1 '
           f'--gradient_checkpointing '
           f'--mixed_precision="fp16"  '
           f'--max_train_steps=500 '
           f'--lr=1e-05 '
           f'--max_grad_norm=1 '
           f'--lr_scheduler="constant" '
           f'--lr_warmup_steps=0 '
           f'--output_dir=/content/decoder_prior_saves '
           f'--rank=4 '
           f'--checkpointing_steps=500')

In [41]:
!{command}

2024-05-08 15:12:53.412787: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 15:12:53.412839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 15:12:53.414298: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 15:12:54.579936: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_

In [42]:
import sys
from diffusers import KandinskyV22Pipeline, KandinskyV22PriorPipeline
import torch
import PIL
import torch
from diffusers.utils import load_image
from torchvision import transforms
from transformers import CLIPVisionModelWithProjection
from diffusers.models import UNet2DConditionModel
import numpy as np

/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [43]:
image_encoder = CLIPVisionModelWithProjection.from_pretrained('kandinsky-community/kandinsky-2-2-prior', subfolder='image_encoder').to(torch.float16).to('cuda')
unet = UNet2DConditionModel.from_pretrained('kandinsky-community/kandinsky-2-2-decoder', subfolder='unet').to(torch.float16).to('cuda')
prior = KandinskyV22PriorPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-prior', image_encoder=image_encoder, torch_dtype=torch.float16)
prior = prior.to("cuda")
decoder = KandinskyV22Pipeline.from_pretrained('kandinsky-community/kandinsky-2-2-decoder', unet=unet, torch_dtype=torch.float16)
decoder = decoder.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model_index.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

model_index.json:   0%|          | 0.00/250 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [52]:
from diffusers.models.attention_processor import LoRAAttnProcessor, LoRAAttnAddedKVProcessor
lora_attn_procs = {}
d = torch.load('/Users/krvzk/Downloads/pytorch_model.bin')
for name in decoder.unet.attn_processors.keys():
    cross_attention_dim = None if name.endswith("attn1.processor") else decoder.unet.config.cross_attention_dim
    if name.startswith("mid_block"):
        hidden_size = decoder.unet.config.block_out_channels[-1]
    elif name.startswith("up_blocks"):
        block_id = int(name[len("up_blocks.")])
        hidden_size = list(reversed(decoder.unet.config.block_out_channels))[block_id]
    elif name.startswith("down_blocks"):
        block_id = int(name[len("down_blocks.")])
        hidden_size = decoder.unet.config.block_out_channels[block_id]
    lora_attn_procs[name] = LoRAAttnAddedKVProcessor(
            hidden_size=hidden_size,
            cross_attention_dim=cross_attention_dim,
            rank=4,
    ).to('cuda')

decoder.unet.set_attn_processor(lora_attn_procs)
decoder.unet.load_state_dict(d, strict=False)
None

FileNotFoundError: [Errno 2] No such file or directory: '/Users/krvzk/Downloads/pytorch_model.bin'

In [ ]:
from diffusers.models.attention_processor import LoRAAttnProcessor, LoRAAttnAddedKVProcessor
lora_attn_procs = {}
for name in prior.prior.attn_processors.keys():
    lora_attn_procs[name] = LoRAAttnProcessor(hidden_size=2048).to('cuda')
prior.prior.set_attn_processor(lora_attn_procs)
prior.prior.load_state_dict(torch.load('/content/decoder_prior_saves/checkpoint-500/pytorch_model.bin'), strict=False)
None

In [ ]:
negative_prior_prompt ='lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature'
torch.manual_seed(42)
img_emb = prior(prompt='A robot pokemon, 4k photo', num_inference_steps=25, num_images_per_prompt=1,)
negative_emb = prior(prompt=negative_prior_prompt, num_inference_steps=25, num_images_per_prompt=1)
images = decoder(image_embeds=img_emb.image_embeds, negative_image_embeds=negative_emb.image_embeds, num_inference_steps=50, height=512, width=512)


In [ ]:
images.images[0]